# Import

In [1]:
import math
import random
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from spellchecker import SpellChecker

stop_words = set(stopwords.words('english')) 

# 단어 교체(Word Replacement)

In [2]:
spell = SpellChecker()


def get_word_replacement(sentence):
    cnt = 0
    candidate_word = {}
    while True:
        for origin_word in word_tokenize(sentence):
            if origin_word not in stop_words:
                word = origin_word
                # find those words that may be misspelled
                word = word.replace(word[random.randrange(0,len(word))], '')
                word_dict = {}

                candid = spell.candidates(word)
                if candid:
                    # Get a list of `likely` options
                    for cand in candid:
                        word_dict[cand] = spell.word_usage_frequency(cand)
                    likely = min(word_dict.items())
                    candidate_word[likely[0]] = (likely[1], origin_word)

        if candidate_word:
            minimum = min(candidate_word)
            return sentence.replace(candidate_word[minimum][1], minimum)
        else:
            if cnt >= 3:
                return sentence
            cnt += 1

get_similar_syllable(sentence)

NameError: name 'get_similar_syllable' is not defined

# 유사 발음(Similar Sound)

In [3]:
from SoundsLike.SoundsLike import Search
"""
Takes a search term, searches its pronunciation,
and returns a list of end-rhyming words in CMU Dict.
End-rhyming words are defined as sharing the same phones from the last stressed syllable on.
Optional arguments include:
    match_syllables (bool):
        If True, returns only words with the same number of syllables as Search_Term.
        If unspecified, defaults to match_syllables=False.
    match_alpha (bool):
        If True, returns only words with the same first letter as Search_Term.
        If unspecified, defaults to match_alpha=False.
"""
#sentence = "the lion king"
sentence = "The Dark Knight"
def get_similar_sound(sentence):
    candidate_word = {}
    m_candidate_word = {}
    for word in word_tokenize(sentence):
        sim_words = []
        if word not in stop_words:
            try:
                sim_words = Search.endRhymes(word, match_syllables=True, match_alpha=True, generate=False)
                random.shuffle(sim_words)
                if sim_words:
                    for sim_word in sim_words:
                        if sim_word.lower() != word.lower():
                            m_candidate_word[word] = sim_word.lower()
            except:
                pass
            
            else:
                try:
                    sim_words = Search.endRhymes(word, match_syllables=True, match_alpha=False, generate=False)
                    sim_word = sim_words[random.randrange(0, len(sim_words))]
                    candidate_word[word] = sim_word
                except:
                    pass
                
    if m_candidate_word:
        for mcw in m_candidate_word.keys():
            return sentence.replace(mcw, m_candidate_word[mcw])
    elif candidate_word:
        for cw in candidate_word.keys():
            return sentence.replace(cw, candidate_word[cw])
    else:
        return sentence
get_similar_sound(sentence)


'The darke Knight'

# 유사 음절(Similar Syllable)

In [4]:
from Phyme import Phyme

ph = Phyme()

sentence = "The Dark Knight"
#sentence = "captain america"
def get_similar_syllable(sentence):
    candidate_word = {}
    for word in word_tokenize(sentence):
        # find rhymes with the same vowels and some of the same consonants, with some swapped out for other consonants. FACTOR -> FASTER
        try:
            rhymes = ph.get_substitution_rhymes(word)
            for rhyme in rhymes.items():
                for r in rhyme[1]:
                    if word.lower() in r.lower():
                        if word.lower() not in stop_words and word.lower() != r.lower():
                            candidate_word[word] = r
        except:
            pass
    
    if candidate_word:
        for cw in candidate_word.keys():
            return sentence.replace(cw, candidate_word[cw])
    else:
        return sentence
get_similar_syllable(sentence)


'The darke Knight'

# Data Preprocessing

In [5]:
#df = pd.read_csv("/home/synpjh/dayeon/dayeon4/humor_titles.csv")
df = pd.read_csv("/home/synpjh/dayeon/dayeon4/popular_movie_titles.csv")
df = df.sort_values(by=['votes'], ascending=False)
df.head()

,movie title,votes
329,The Shawshank Redemption,1965648
758,The Dark Knight,1937000
1105,Inception,1720760
1027,Fight Club,1575298
580,Pulp Fiction,1536659


In [18]:
def get_humor_title(f, col):
    humor_title = []
    cnt = 0
    for i in range(len(df)):   

        if col.iloc[i].lower() != f(col.iloc[i]).lower():
            humor_title.append( (col.iloc[i], f(col.iloc[i])) )
            cnt += 1
        if cnt >= 63:
            return humor_title
    return humor_title

In [23]:
ssd = get_humor_title(get_similar_sound, df['movie title'])
ssd

[('The Shawshank Redemption', 'The sawshank Redemption'),
 ('The Dark Knight', 'The darke Knight'),
 ('Fight Club', 'fright Club'),
 ('Pulp Fiction', 'Pulp friction'),
 ('Forrest Gump', 'Forrest gumpp'),
 ('The Lord of the Rings: The Fellowship of the Ring',
  "The Lord of the ring's: The Fellowship of the Ring"),
 ('The Lord of the Rings: The Return of the King',
  "The Lord of the ring's: The Return of the King"),
 ('The Godfather', 'The grandfather'),
 ('Game of Thrones', 'Aim of Thrones'),
 ('The Dark Knight Rises', 'The darke Knight Rises'),
 ('The Lord of the Rings: The Two Towers',
  "The Lord of the ring's: The Two Towers"),
 ('Gladiator', 'generator'),
 ('Batman Begins', 'bethann Begins'),
 ('Breaking Bad', 'baking Bad'),
 ('Star Wars: Episode IV - A New Hope', 'spahr Wars: Episode IV - A New Hope'),
 ('The Silence of the Lambs', "The Silence of the lamb's"),
 ('Saving Private Ryan', 'shaving Private Ryan'),
 ("Schindler's List", "shindler's List"),
 ('Avatar', 'alcazar'),
 ('

In [24]:
ssd16 = ssd[:16]
ssd16

[('The Shawshank Redemption', 'The sawshank Redemption'),
 ('The Dark Knight', 'The darke Knight'),
 ('Fight Club', 'fright Club'),
 ('Pulp Fiction', 'Pulp friction'),
 ('Forrest Gump', 'Forrest gumpp'),
 ('The Lord of the Rings: The Fellowship of the Ring',
  "The Lord of the ring's: The Fellowship of the Ring"),
 ('The Lord of the Rings: The Return of the King',
  "The Lord of the ring's: The Return of the King"),
 ('The Godfather', 'The grandfather'),
 ('Game of Thrones', 'Aim of Thrones'),
 ('The Dark Knight Rises', 'The darke Knight Rises'),
 ('The Lord of the Rings: The Two Towers',
  "The Lord of the ring's: The Two Towers"),
 ('Gladiator', 'generator'),
 ('Batman Begins', 'bethann Begins'),
 ('Breaking Bad', 'baking Bad'),
 ('Star Wars: Episode IV - A New Hope', 'spahr Wars: Episode IV - A New Hope'),
 ('The Silence of the Lambs', "The Silence of the lamb's")]

In [25]:
ssd32 = ssd[:32]
ssd32

[('The Shawshank Redemption', 'The sawshank Redemption'),
 ('The Dark Knight', 'The darke Knight'),
 ('Fight Club', 'fright Club'),
 ('Pulp Fiction', 'Pulp friction'),
 ('Forrest Gump', 'Forrest gumpp'),
 ('The Lord of the Rings: The Fellowship of the Ring',
  "The Lord of the ring's: The Fellowship of the Ring"),
 ('The Lord of the Rings: The Return of the King',
  "The Lord of the ring's: The Return of the King"),
 ('The Godfather', 'The grandfather'),
 ('Game of Thrones', 'Aim of Thrones'),
 ('The Dark Knight Rises', 'The darke Knight Rises'),
 ('The Lord of the Rings: The Two Towers',
  "The Lord of the ring's: The Two Towers"),
 ('Gladiator', 'generator'),
 ('Batman Begins', 'bethann Begins'),
 ('Breaking Bad', 'baking Bad'),
 ('Star Wars: Episode IV - A New Hope', 'spahr Wars: Episode IV - A New Hope'),
 ('The Silence of the Lambs', "The Silence of the lamb's"),
 ('Saving Private Ryan', 'shaving Private Ryan'),
 ("Schindler's List", "shindler's List"),
 ('Avatar', 'alcazar'),
 ('

In [26]:
ssd64 = ssd[:64]
ssd64

[('The Shawshank Redemption', 'The sawshank Redemption'),
 ('The Dark Knight', 'The darke Knight'),
 ('Fight Club', 'fright Club'),
 ('Pulp Fiction', 'Pulp friction'),
 ('Forrest Gump', 'Forrest gumpp'),
 ('The Lord of the Rings: The Fellowship of the Ring',
  "The Lord of the ring's: The Fellowship of the Ring"),
 ('The Lord of the Rings: The Return of the King',
  "The Lord of the ring's: The Return of the King"),
 ('The Godfather', 'The grandfather'),
 ('Game of Thrones', 'Aim of Thrones'),
 ('The Dark Knight Rises', 'The darke Knight Rises'),
 ('The Lord of the Rings: The Two Towers',
  "The Lord of the ring's: The Two Towers"),
 ('Gladiator', 'generator'),
 ('Batman Begins', 'bethann Begins'),
 ('Breaking Bad', 'baking Bad'),
 ('Star Wars: Episode IV - A New Hope', 'spahr Wars: Episode IV - A New Hope'),
 ('The Silence of the Lambs', "The Silence of the lamb's"),
 ('Saving Private Ryan', 'shaving Private Ryan'),
 ("Schindler's List", "shindler's List"),
 ('Avatar', 'alcazar'),
 ('

In [27]:
sse = get_humor_title(get_similar_syllable, df['movie title'])
sse

[('The Dark Knight', 'The darke Knight'),
 ('Fight Club', 'Fight clubb'),
 ('Pulp Fiction', 'Pulp nonfiction'),
 ('Forrest Gump', 'Forrest gumpp'),
 ('The Lord of the Rings: The Fellowship of the Ring',
  'The Lord of the springs(1): The Fellowship of the Ring'),
 ('The Lord of the Rings: The Return of the King',
  'The Lord of the springs(1): The Return of the King'),
 ('Game of Thrones', 'postgame of Thrones'),
 ('The Dark Knight Rises', 'The darke Knight Rises'),
 ('The Lord of the Rings: The Two Towers',
  'The Lord of the springs(1): The Two Towers'),
 ('Breaking Bad', 'Breaking shabad'),
 ('Star Wars: Episode IV - A New Hope',
  'superstar Wars: Episode IV - A New Hope'),
 ('The Silence of the Lambs', "The Silence of the lambs'"),
 ('Saving Private Ryan', 'Saving Private obryan'),
 ("Schindler's List", "Schindlero'shell List"),
 ('Star Wars: Episode V - The Empire Strikes Back',
  'superstar Wars: Episode V - The Empire Strikes Back'),
 ('The Wolf of Wall Street', 'The dewolfe of

In [28]:
sse16 = sse[:16]
sse16

[('The Dark Knight', 'The darke Knight'),
 ('Fight Club', 'Fight clubb'),
 ('Pulp Fiction', 'Pulp nonfiction'),
 ('Forrest Gump', 'Forrest gumpp'),
 ('The Lord of the Rings: The Fellowship of the Ring',
  'The Lord of the springs(1): The Fellowship of the Ring'),
 ('The Lord of the Rings: The Return of the King',
  'The Lord of the springs(1): The Return of the King'),
 ('Game of Thrones', 'postgame of Thrones'),
 ('The Dark Knight Rises', 'The darke Knight Rises'),
 ('The Lord of the Rings: The Two Towers',
  'The Lord of the springs(1): The Two Towers'),
 ('Breaking Bad', 'Breaking shabad'),
 ('Star Wars: Episode IV - A New Hope',
  'superstar Wars: Episode IV - A New Hope'),
 ('The Silence of the Lambs', "The Silence of the lambs'"),
 ('Saving Private Ryan', 'Saving Private obryan'),
 ("Schindler's List", "Schindlero'shell List"),
 ('Star Wars: Episode V - The Empire Strikes Back',
  'superstar Wars: Episode V - The Empire Strikes Back'),
 ('The Wolf of Wall Street', 'The dewolfe of

In [29]:
sse32 = sse[:32]
sse32

[('The Dark Knight', 'The darke Knight'),
 ('Fight Club', 'Fight clubb'),
 ('Pulp Fiction', 'Pulp nonfiction'),
 ('Forrest Gump', 'Forrest gumpp'),
 ('The Lord of the Rings: The Fellowship of the Ring',
  'The Lord of the springs(1): The Fellowship of the Ring'),
 ('The Lord of the Rings: The Return of the King',
  'The Lord of the springs(1): The Return of the King'),
 ('Game of Thrones', 'postgame of Thrones'),
 ('The Dark Knight Rises', 'The darke Knight Rises'),
 ('The Lord of the Rings: The Two Towers',
  'The Lord of the springs(1): The Two Towers'),
 ('Breaking Bad', 'Breaking shabad'),
 ('Star Wars: Episode IV - A New Hope',
  'superstar Wars: Episode IV - A New Hope'),
 ('The Silence of the Lambs', "The Silence of the lambs'"),
 ('Saving Private Ryan', 'Saving Private obryan'),
 ("Schindler's List", "Schindlero'shell List"),
 ('Star Wars: Episode V - The Empire Strikes Back',
  'superstar Wars: Episode V - The Empire Strikes Back'),
 ('The Wolf of Wall Street', 'The dewolfe of

In [30]:
sse64 = sse[:64]
sse64

[('The Dark Knight', 'The darke Knight'),
 ('Fight Club', 'Fight clubb'),
 ('Pulp Fiction', 'Pulp nonfiction'),
 ('Forrest Gump', 'Forrest gumpp'),
 ('The Lord of the Rings: The Fellowship of the Ring',
  'The Lord of the springs(1): The Fellowship of the Ring'),
 ('The Lord of the Rings: The Return of the King',
  'The Lord of the springs(1): The Return of the King'),
 ('Game of Thrones', 'postgame of Thrones'),
 ('The Dark Knight Rises', 'The darke Knight Rises'),
 ('The Lord of the Rings: The Two Towers',
  'The Lord of the springs(1): The Two Towers'),
 ('Breaking Bad', 'Breaking shabad'),
 ('Star Wars: Episode IV - A New Hope',
  'superstar Wars: Episode IV - A New Hope'),
 ('The Silence of the Lambs', "The Silence of the lambs'"),
 ('Saving Private Ryan', 'Saving Private obryan'),
 ("Schindler's List", "Schindlero'shell List"),
 ('Star Wars: Episode V - The Empire Strikes Back',
  'superstar Wars: Episode V - The Empire Strikes Back'),
 ('The Wolf of Wall Street', 'The dewolfe of